In [1]:
#git clone https://github.com/likr/kplib.git

In [1]:
import os
import time
import glob
import pandas as pd
from ortools.algorithms.python import knapsack_solver

# Thời gian giới hạn cho mỗi bài test (giây)
TIME_LIMIT_SECONDS = 160

def read_kp_file(file_path):
    """Đọc file test case Knapsack."""
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Đọc số lượng items và sức chứa
    n = int(lines[1].strip())
    capacity = int(lines[2].strip())
    
    # Đọc trọng lượng và giá trị
    weights = []
    values = []
    for i in range(4, 4 + n):
        parts = lines[i].strip().split()
        values.append(int(parts[0]))
        weights.append(int(parts[1]))
    
    return n, capacity, values, weights

def solve_knapsack(file_path):
    """Giải bài toán Knapsack cho một file test case."""
    try:
        n, capacity, values, weights_list = read_kp_file(file_path)
        weights = [weights_list]  # OR-Tools yêu cầu list của lists
        capacities = [capacity]
        
        # Tạo solver
        solver = knapsack_solver.KnapsackSolver(
            knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
            "KnapsackSolver"
        )
        
        # Thiết lập thời gian giới hạn
        solver.set_time_limit(TIME_LIMIT_SECONDS)
        
        # Khởi tạo solver với dữ liệu
        solver.init(values, weights, capacities)
        
        # Bắt đầu tính thời gian
        start_time = time.time()
        
        # Giải bài toán
        computed_value = solver.solve()
        
        # Kết thúc tính thời gian
        elapsed_time = time.time() - start_time
        
        # Xác định tính tối ưu của lời giải
        # Nếu thời gian chạy đã đạt tới giới hạn, coi là không tối ưu
        is_optimal = elapsed_time < (TIME_LIMIT_SECONDS - 0.1)
        
        
        # Tìm các items được đưa vào túi và tính tổng trọng lượng
        packed_items = []
        total_weight = 0
        for i in range(len(values)):
            if solver.best_solution_contains(i):
                packed_items.append(i)
                total_weight += weights[0][i]
        
        return {
            'testcase': os.path.relpath(file_path, os.path.join(os.getcwd(), 'kplib')),
            'Nitems': n,
            'value': computed_value,
            'weight': total_weight,
            'time': f"{elapsed_time:.6f}",
            'optimal': is_optimal
        }
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return {
            'testcase': os.path.relpath(file_path, os.path.join(os.getcwd(), 'kplib')),
            'Nitems': -1,
            'value': -1,
            'weight': -1,
            'time': -1,
            'optimal': False
        }

def find_test_cases(kplib_dir):
    """Tìm các test cases từ các nhóm khác nhau và kích thước khác nhau."""
    # Kích thước items cần lấy
    item_sizes = ['00050', '00100', '00200', '00500', '01000', '02000', '05000', '10000']
    
    # Danh sách test cases được chọn
    selected_test_cases = []
    
    # Lặp qua các nhóm từ 00 đến 12
    for group_num in range(13):
        group_name = f"{group_num:02d}"
        
        # Xác định tên đầy đủ của thư mục nhóm
        group_paths = []
        for folder in os.listdir(kplib_dir):
            if folder.startswith(group_name):
                group_paths.append(os.path.join(kplib_dir, folder))
        
        if not group_paths:
            print(f"Warning: No folder found for group {group_name}")
            continue
            
        group_dir = group_paths[0]
        
        # Lặp qua các kích thước
        for size in item_sizes:
            size_path = os.path.join(group_dir, "n" + size)
            
            # Kiểm tra thư mục kích thước có tồn tại
            if not os.path.exists(size_path):
                continue
                
            # Tìm các instances trong R01000 hoặc R10000
            r_folders = [f for f in os.listdir(size_path) if f.startswith("R")]
            
            # Lấy một test case từ mỗi R folder
            for r_folder in r_folders:
                r_path = os.path.join(size_path, r_folder)
                if os.path.isdir(r_path):
                    # Lấy file 16.kp từ thư mục R
                    kp_files = [f for f in os.listdir(r_path) if f.endswith(".kp")]
                    if kp_files:
                        test_file = os.path.join(r_path, kp_files[16])
                        selected_test_cases.append(test_file)
    
    return selected_test_cases


def main():
    print("Starting Knapsack Problem Solver with OR-Tools...")
    
    # Đường dẫn đến thư mục kplib
    kplib_dir = os.path.join(os.getcwd(), 'kplib')
    
    # Kiểm tra xem thư mục kplib có tồn tại không
    if not os.path.exists(kplib_dir):
        print(f"Error: kplib directory not found at {kplib_dir}")
        return
    
    print(f"Found kplib directory at {kplib_dir}")
    
    # Tìm test cases
    test_cases = find_test_cases(kplib_dir)
    print(f"Found {len(test_cases)} test cases")
    
    # Kiểm tra nếu không tìm thấy test cases nào
    if not test_cases:
        print("No test cases found. Please check if kplib is properly cloned.")
        return
    
    # Kết quả
    results = []
    
    # Giải từng test case
    for i, test_file in enumerate(test_cases):
        print(f"Processing test case {i+1}/{len(test_cases)}: {os.path.basename(test_file)}")
        result = solve_knapsack(test_file)
        results.append(result)
        
        # In kết quả trung gian
        print(f"  Value: {result['value']}, Weight: {result['weight']}, Time: {result['time']}s, Optimal: {result['optimal']}")
    
    # Tạo DataFrame từ kết quả
    results_df = pd.DataFrame(results)
    
    # Lưu kết quả vào file CSV
    results_df.to_csv('knapsack_results.csv', index=False)
    print("Results saved to knapsack_results.csv")
    

    
    # In báo cáo trong terminal
    print("\n===== KNAPSACK TEST RESULTS =====")
    print(results_df)
    

if __name__ == "__main__":
    main()

load c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\ortools.dll...
Starting Knapsack Problem Solver with OR-Tools...
Found kplib directory at d:\NĂM 2 KÌ 2\AI\ORtool\kplib
Found 208 test cases
Processing test case 1/208: s016.kp
  Value: 19232, Weight: 10474, Time: 0.000000s, Optimal: True
Processing test case 2/208: s